### Import Libraries

In [16]:
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from keras.models import load_model
from matplotlib.patches import Circle
import cv2
import numpy as np
from sklearn.preprocessing import Normalizer
from scipy.spatial.distance import cosine
import os
import datetime
import xlwt
import xlrd
from xlwt import Workbook 
from xlutils.copy import copy
import time
# from facenet_pytorch import MTCNN
# from cv2 import dnn_superres

### Load model of MTCNN and FaceNet

In [17]:
encoder_model = './model/facenet_keras.h5'

In [18]:
detector=MTCNN()
face_encoder = load_model(encoder_model)
people_dir = './people'
encoding_dict = dict()

In [19]:
classNames = []
with open('coco.names','r') as f:
    classNames = f.read().splitlines()
print(classNames)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [31]:
weightsPath = "frozen_inference_graph.pb"
configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 0x7f8dac3cf150>

In [37]:
# sr = cv2.dnn_superres.DnnSuperResImpl_create()
# path = "LapSRN_x8.pb"
# sr.readModel(path)

# # Set the desired model and scale to get correct pre- and post-processing
# sr.setModel("lapsrn", 8)

# sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

### Encoding of face using FaceNet

In [22]:
def get_encode(face_encoder, face, size):
    face = normalize(face)
    face = cv2.resize(face, size)
    encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
    return encode

### Extract Face for encoding

In [23]:
def get_face(img, box):
    x1, y1, width, height = box
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = img[y1:y2, x1:x2]
    return face, (x1, y1), (x2, y2)

In [24]:
def normalize(img):
    mean, std = img.mean(), img.std()
    return (img - mean) / std

In [25]:
l2_normalizer = Normalizer('l2')

In [26]:
def export(a):
    

    ct = datetime.datetime.now()
    
    ws = xlrd.open_workbook('names_list.xls',formatting_info=True)
    sheet = ws.sheet_by_index(0)
    wb=copy(ws)
    names_sheet=wb.get_sheet(0)
    row_s=sheet.nrows
    col_s=sheet.ncols
    
    names_sheet.write(0,col_s,str(ct.year)+"_"+str(ct.month)+"_"+str(ct.day)+"_"+str(ct.hour))
    
    for i in range(1,row_s):   
        
        if sheet.cell_value(i,0) in a:
            names_sheet.write(i, col_s,"P")
                
        else:
                
            names_sheet.write(i, col_s,"A")
    wb.save("names_list.xls") 

In [27]:
def mark_attendance(a):
    

    ct = datetime.datetime.now()
    
    


    workbook = xlwt.Workbook()  

    sheet = workbook.add_sheet(str(ct.year)+"_"+str(ct.month)+"_"+str(ct.day)) 
    sheet.write(0,0,"Name")


    row = 1
    col = 0
    if len(a)>0:
        for person_name in os.listdir(people_dir):
            print(person_name)
            print(a)

            for x in range(0,len(a)):
                if str(person_name) in a:
                    sheet.write(row, col,     str(a[x]))
                    sheet.write(row,col+1,"P")
                    
                else:
                    sheet.write(row, col,     str(a[x]),)
                    sheet.write(row,col+1,"A")
                row+=1
         
            
  
        workbook.save("sample_class_1.xls") 
    else:
        sheet.write(1,0,"No one is present")
        workbook.save("sample_class_1.xls") 


### Saved Images whhich are encoded and stored in a Dictionary

In [28]:
for person_name in os.listdir(people_dir):
    person_dir = os.path.join(people_dir, person_name)
    encodes = []
    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        results = detector.detect_faces(img)
#         results, probs= detector.detect(img)
        
        
        
        if results:
            res = max(results, key=lambda b: b['box'][2] * b['box'][3])
            face, _, _ = get_face(img, res['box'])
            
            face = normalize(face)
            face = cv2.resize(face,(160,160))
            encode = face_encoder.predict(np.expand_dims(face, axis=0))[0]
            encodes.append(encode)
    if encodes:
        encode = np.sum(encodes, axis=0)
        encode = l2_normalizer.transform(np.expand_dims(encode, axis=0))[0]
        encoding_dict[person_name] = encode  

In [32]:
recognition_t=0.6
confidence_t=0.99
thres = 0.5 # Threshold to detect object
nms_threshold = 0.2 #(0.1 to 1) 1 means no suppress , 0.1 means high suppress
font = cv2.FONT_HERSHEY_PLAIN
#font = cv2.FONT_HERSHEY_COMPLEX
Colors = np.random.uniform(0, 255, size=(len(classNames), 3))

### Live Face Detection and Recognition (Press q to end video stream)

In [48]:
video =cv2.VideoCapture(0)
present_candidates=[]

fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0



while True:
    check,frame=video.read()
    total_frames = total_frames + 1
#     frame=sr.upsample(frame)
    
    faces=detector.detect_faces(frame)
    
    classIds, confs, bbox = net.detect(frame,confThreshold=thres)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1,-1)[0])
    confs = list(map(float,confs))
    
    indices = cv2.dnn.NMSBoxes(bbox,confs,thres,nms_threshold)
    
    if len(classIds) != 0:
        
        for i in indices:
            i = i[0]
            box = bbox[i]
            confidence = str(round(confs[i],2))
            color = Colors[classIds[i][0]-1]
            x,y,w,h = box[0],box[1],box[2],box[3]
            cv2.rectangle(frame, (x,y), (x+w,y+h), color, thickness=2)
            cv2.putText(frame, classNames[classIds[i][0]-1]+" "+confidence,(x+10,y+20),
                        font,1,color,2)
    
    if faces !=[]:
       
        for person in faces:
            bounding_box=person["box"]
            keypoints=person["keypoints"]
# #             if person['confidence'] < confidence_t:
# #                 continue
            face, pt_1, pt_2 = get_face(frame, person['box'])
            cv2.rectangle(frame, pt_1 , pt_2, (0, 255, 0), 2)
            encode = get_encode(face_encoder, face,(160,160))
            encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
            name = 'unknown'
            
            
            distance = float("inf")
            
            
            for (db_name, db_enc) in encoding_dict.items():
        
                dist = cosine(db_enc, encode)
          
                if dist < recognition_t and dist < distance:
                
                    name = db_name
                    distance = dist
                    if name not in present_candidates:
                        present_candidates.append(name)
                    
                    
            if name == 'unknown':
                cv2.rectangle(frame, pt_1, pt_2, (0, 0, 255), 2)
                cv2.putText(frame,name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
                
            else:
    
                cv2.rectangle(frame, pt_1 , pt_2, (0, 255, 0), 2)
                cv2.putText(frame,name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 200, 200), 2)

                

    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames / time_diff.seconds)

    fps_text = "FPS: {:.2f}".format(fps)

    cv2.putText(frame, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    
    cv2.imshow('frame',frame)  
    if cv2.waitKey(5) & 0xFF == ord('q'):
          break
            
            
            
video.release()
cv2.destroyAllWindows()

Classs Ids=== [[1]]
Faces===== [{'box': [279, 188, 185, 242], 'confidence': 0.9999814033508301, 'keypoints': {'left_eye': (337, 280), 'right_eye': (423, 284), 'nose': (379, 332), 'mouth_left': (338, 373), 'mouth_right': (413, 376)}}]
Classs Ids=== [[1]]
Faces===== [{'box': [282, 189, 183, 235], 'confidence': 0.9999909400939941, 'keypoints': {'left_eye': (338, 280), 'right_eye': (424, 282), 'nose': (382, 331), 'mouth_left': (342, 372), 'mouth_right': (416, 375)}}]
Classs Ids=== [[1]]
Faces===== [{'box': [284, 191, 181, 229], 'confidence': 0.9999995231628418, 'keypoints': {'left_eye': (339, 278), 'right_eye': (426, 281), 'nose': (383, 327), 'mouth_left': (342, 368), 'mouth_right': (415, 372)}}]
Classs Ids=== [[1]]
Faces===== [{'box': [288, 193, 180, 222], 'confidence': 0.9999995231628418, 'keypoints': {'left_eye': (341, 278), 'right_eye': (425, 280), 'nose': (383, 324), 'mouth_left': (345, 367), 'mouth_right': (416, 370)}}]
Classs Ids=== [[1]]
Faces===== [{'box': [285, 190, 178, 225], 'c

Classs Ids=== [[ 1]
 [72]]
Faces===== [{'box': [268, 209, 173, 218], 'confidence': 0.9999617338180542, 'keypoints': {'left_eye': (327, 293), 'right_eye': (407, 294), 'nose': (370, 337), 'mouth_left': (332, 378), 'mouth_right': (399, 380)}}, {'box': [548, 269, 37, 44], 'confidence': 0.9649778008460999, 'keypoints': {'left_eye': (558, 285), 'right_eye': (575, 283), 'nose': (567, 293), 'mouth_left': (563, 304), 'mouth_right': (576, 301)}}]
Classs Ids=== [[ 1]
 [72]]
Faces===== [{'box': [271, 203, 174, 224], 'confidence': 0.9998688697814941, 'keypoints': {'left_eye': (326, 289), 'right_eye': (409, 290), 'nose': (368, 337), 'mouth_left': (334, 378), 'mouth_right': (400, 380)}}, {'box': [547, 232, 38, 44], 'confidence': 0.9836770296096802, 'keypoints': {'left_eye': (558, 247), 'right_eye': (577, 248), 'nose': (567, 255), 'mouth_left': (558, 264), 'mouth_right': (574, 265)}}, {'box': [554, 269, 37, 43], 'confidence': 0.9448764324188232, 'keypoints': {'left_eye': (564, 285), 'right_eye': (580,

Classs Ids=== [[ 1]
 [77]]
Faces===== [{'box': [298, 105, 173, 223], 'confidence': 0.9997124075889587, 'keypoints': {'left_eye': (358, 191), 'right_eye': (442, 193), 'nose': (404, 239), 'mouth_left': (364, 281), 'mouth_right': (431, 283)}}]
Classs Ids=== [[ 1]
 [77]]
Faces===== [{'box': [297, 103, 173, 220], 'confidence': 0.9999494552612305, 'keypoints': {'left_eye': (358, 187), 'right_eye': (439, 191), 'nose': (401, 231), 'mouth_left': (359, 273), 'mouth_right': (428, 277)}}]
Classs Ids=== [[ 1]
 [75]
 [77]]
Faces===== [{'box': [302, 136, 141, 175], 'confidence': 0.9999613761901855, 'keypoints': {'left_eye': (348, 204), 'right_eye': (414, 207), 'nose': (383, 235), 'mouth_left': (353, 273), 'mouth_right': (406, 276)}}]
Classs Ids=== [[ 1]
 [75]
 [77]]
Faces===== [{'box': [306, 141, 140, 184], 'confidence': 0.9996354579925537, 'keypoints': {'left_eye': (352, 210), 'right_eye': (419, 214), 'nose': (387, 248), 'mouth_left': (354, 283), 'mouth_right': (410, 286)}}]
Classs Ids=== [[ 1]
 [75

In [49]:
count=0
for no,obj in enumerate(classIds):
    if obj==[1]:
        count+=1
        
        
print(count)#####Number of people detected 

1


### Snap based Face Detection and recognition (Press Spacebar to take a snap and Escape to end the stream)

In [51]:
video1 =cv2.VideoCapture(0)
present_candidates_1=[]
img_counter=0
fps_start_time = datetime.datetime.now()
fps = 0
total_frames = 0
while True:
    
    check,frame=video1.read()
    total_frames = total_frames + 1
    
    
    fps_end_time = datetime.datetime.now()
    time_diff = fps_end_time - fps_start_time
    if time_diff.seconds == 0:
        fps = 0.0
    else:
        fps = (total_frames / time_diff.seconds)

    fps_text = "FPS: {:.2f}".format(fps)

    cv2.putText(frame, fps_text, (5, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    
    cv2.imshow("SS",frame)
    k=cv2.waitKey(1)
    
    if k%256 == 27:
        print("Escape")
        break
        
    elif k%256==32:
        faces=detector.detect_faces(frame)
        if faces !=[]:
            for person in faces:
                bounding_box=person["box"]
                keypoints=person["keypoints"]
    #             if person['confidence'] < confidence_t:
    #                 continue
                face, pt_1, pt_2 = get_face(frame, person['box'])
                encode = get_encode(face_encoder, face,(160,160))
                encode = l2_normalizer.transform(encode.reshape(1, -1))[0]
                name = 'unknown'


                distance = float("inf")


                for (db_name, db_enc) in encoding_dict.items():

                    dist = cosine(db_enc, encode)

                    if dist < recognition_t and dist < distance:

                        name = db_name
                        distance = dist
                        if name not in present_candidates_1:
                            present_candidates_1.append(name)

                if name == 'unknown':
                    cv2.rectangle(frame, pt_1, pt_2, (0, 0, 255), 2)
                    cv2.putText(frame,name, pt_1, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

                else:
                    cv2.rectangle(frame, pt_1, pt_2, (0, 255, 0), 2)
                    cv2.putText(frame,name + f'__{distance:.2f}', (pt_1[0], pt_1[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 200, 200), 2)
                    
        
                    
        img_name="opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name,frame)
        print("Screenshot")
        img_counter+=1

video1.release()
cv2.destroyAllWindows()

Escape


### Candidates present (Live video)

In [14]:
present_candidates

['Aditya']

### Candidates present (Snap)

In [52]:
present_candidates_1

[]

In [17]:
export(present_candidates_1)

In [16]:
export(present_candidates)

In [46]:
len(faces)#Number of faces detected

2